Winter 2024

Analyzing Drug Reviews

Web Scrapping part 1

Website used: https://www.webmd.com/drugs/2/index

In [ ]:
#Importing necessary libraries
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
from tqdm import tqdm

In [ ]:
#first to check the process we will use one drug:

#acetaminophen
url = "https://reviews.webmd.com/drugs/drugreview-362-acetaminophen-oral"

In [ ]:
response = requests.get(url)

In [ ]:
response = response.content

In [ ]:
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
finder = soup.find('div')
reviews = finder.find_all('div', class_='review-details-holder')

for review in reviews:
    # Extract overall rating
    rate = review.find('div', class_='overall-rating').text.strip()
    overall_rating = float(rate.split()[2])  # Extracting the numeric part as a float

    # Extract review description
    text = review.find('p', class_='description-text').text.strip()

    # Extract Condition
    condition_element = review.find('strong', class_='condition')
    condition = condition_element.text.strip().split('Condition:', 1)[-1].strip() if condition_element else ''

    # Extract Year
    year_element = review.find('div', class_='date')
    month, day, year = map(int, year_element.text.strip().split('/')) if year_element else ''

    # Print the results (this is how the results look like if you dont add them to a csv file)
    print("Overall Rating:", overall_rating)
    print("Review Description:", text)
    print("Condition:", condition)
    print("Year:", year)
    print("---")


Overall Rating: 2.3
Review Description: I have mild rheumatoid arthritis which has been well controlled with methotrexate. I was taken off methotrexate to check my liver enzymes and quickly flared up. Tylenol has been totally ineffective. Celebrex helps some but not for long. I'm back on methotrexate and still not back in control, but somewhat better
Condition: Joint Damage causing Pain and Loss of Function
Year: 2022
---
Overall Rating: 5.0
Review Description: Besides relieving my pain, I FEEL better .. emotionally and mentally. Due to heart condition, this is the only pain reliever my heart doctor recommends. It works great when I take two extra strength. I was told by a medical professional not to exceed 14-16 in a month. Both Ibuprofen and Aspirin interact with my heart meds and specifically I feel a weird heartbeat with ibuprofen.
Condition: Head Pain
Year: 2022
---
Overall Rating: 5.0
Review Description: I took Tylenol Extra Strength Caplets every 6 hours for pain immediately fol

In [ ]:
#now we make a dictionary to loop throgh all of our drugs
#21 drugs

url_dict = {

    "Acetaminophen" : "https://reviews.webmd.com/drugs/drugreview-362-acetaminophen-oral",
    "Ibuprofen" : "https://reviews.webmd.com/drugs/drugreview-5166-ibuprofen-oral",
    "Amoxicillin" : "https://reviews.webmd.com/drugs/drugreview-1531-amoxicillin-oral",
    "Metronidazole" : "https://reviews.webmd.com/drugs/drugreview-6426-metronidazole-oral",
    "Hydroxyzine" : "https://reviews.webmd.com/drugs/drugreview-7681-hydroxyzine-hcl-oral",
    "Clonazepam" : "https://reviews.webmd.com/drugs/drugreview-14403-clonazepam-oral",
    "Azithromycin" : "https://reviews.webmd.com/drugs/drugreview-1527-azithromycin-oral",
    "Tramadol" : "https://reviews.webmd.com/drugs/drugreview-4398-tramadol-oral",
    "Meloxicam" : "https://reviews.webmd.com/drugs/drugreview-911-meloxicam-oral",
    "Methocarbamol" : "https://reviews.webmd.com/drugs/drugreview-8677-methocarbamol-oral",
    "Aspirin" : "https://reviews.webmd.com/drugs/drugreview-1082-aspirin-oral",
    "Alprazolam" : "https://reviews.webmd.com/drugs/drugreview-8171-alprazolam-oral",
    "Morphine" : "https://reviews.webmd.com/drugs/drugreview-327-morphine-oral",
    "Metoprolol Tetrate" : "https://reviews.webmd.com/drugs/drugreview-11207-metoprolol-tartrate-oral",
    "Warfarin" : "https://reviews.webmd.com/drugs/drugreview-3949-warfarin-oral",
    "Codeine Sulfate" : "https://reviews.webmd.com/drugs/drugreview-7420-codeine-sulfate-oral",
    "Penicillin V Potassium" : "https://reviews.webmd.com/drugs/drugreview-8686-penicillin-v-potassium-oral",
    "Metoprolol Tartrate" : "https://reviews.webmd.com/drugs/drugreview-11207-metoprolol-tartrate-oral",
    "Metoprolol Succinate" : "https://reviews.webmd.com/drugs/drugreview-8814-metoprolol-succinate-oral",
    "Famotidine" : "https://reviews.webmd.com/drugs/drugreview-5035-famotidine-oral",
    "Metoclopramide" : "https://reviews.webmd.com/drugs/drugreview-8679-metoclopramide-oral"

}

In [ ]:
""" reviews of each drug are stored in multiple pages. We should find a way to loop through all pages
 for each drug and extract all the reviews. By looking at the url we notice that only the number of page
 changes from page 2 to page n. So we can just change one part of url in each iteration until no next
 page exists. """

' reviews of each drug are stored in multiple pages. We should find a way to loop through all pages\n for each drug and extract all the reviews. By looking at the url we notice that only the number of page\n changes from page 2 to page n. So we can just change one part of url in each iteration until no next\n page exists. '

In [ ]:

csv_file_path = "drug_reviews_sh.csv"

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Drug', 'Review', 'Rating', 'Condition', 'Year']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    total_urls = len(url_dict)
    with tqdm(total=total_urls, desc="Processing URLs", unit="URL") as pbar_urls:
        for name, url in url_dict.items():
            pbar_urls.update(1)  # Increment the outer loop progress bar

            page = 1
            while True:
                current_url = url if page == 1 else f"{url}?page={page}&next_page=true"
                response = requests.get(current_url)
                soup = BeautifulSoup(response.content, 'html.parser')
                finder = soup.find('div')
                reviews = finder.find_all('div', class_='review-details-holder')

                if not reviews:
                    break

                for review in reviews:
                    # Rate
                    rate = review.find('div', class_='overall-rating').text.strip()
                    overall_rating = float(rate.split()[0])

                    # Condition
                    condition_element = review.find('strong', class_='condition')
                    condition = condition_element.text.strip().split('Condition:', 1)[-1].strip() if condition_element else ''

                    #Year
                    year_element = review.find('div', class_='date')
                    month, day, year = map(int, year_element.text.strip().split('/')) if year_element else ''

                    # Review
                    # Check if the element exists
                    description_text_element = review.find('p', class_='description-text')
                    text = description_text_element.text.strip() if description_text_element else ''

                    writer.writerow({'Drug': name, 'Review': text, 'Rating': overall_rating, 'Condition': condition, 'Year': year})

                page += 1

print(f"Data has been written to {csv_file_path}")


Processing URLs: 100%|██████████| 21/21 [10:35<00:00, 30.26s/URL]

Data has been written to drug_reviews_sh.csv


In [ ]:
df = pd.read_csv('/content/drug_reviews_sh (1).csv')
df

,Drug,Review,Rating,Condition,Year
0,Acetaminophen,I have mild rheumatoid arthritis which has bee...,2.3,Joint Damage causing Pain and Loss of Function,2022
1,Acetaminophen,"Besides relieving my pain, I FEEL better .. em...",5.0,Head Pain,2022
2,Acetaminophen,I took Tylenol Extra Strength Caplets every 6 ...,5.0,Pain,2021
3,Acetaminophen,I love this drug. I use it and within 15 minut...,5.0,Painful Periods,2021
4,Acetaminophen,I accidentally took almost 6000 mg of this in ...,5.0,Head Pain,2020
...,...,...,...,...,...
12260,Metoclopramide,I do not like the side effects of this drug.I ...,3.0,Nausea and Vomiting,2008
12261,Metoclopramide,This didn't work for me at all.,2.0,Feel Like Throwing Up,2007
12262,Metoclopramide,forget to take it 30 min. before eating,2.7,Condition in which Stomach Acid is Pushed Into...,2007
12263,Metoclopramide,This medication significantly helps with nause...,3.7,Stomach Muscle Paralysis and Decreased Function,2007


In [ ]:
import shutil

shutil.move("/content/drug_reviews_sh.csv", "/content/drive/MyDrive/NLP_DrugReviews/")


'/content/drive/MyDrive/NLP_DrugReviews/drug_reviews_sh.csv'